# Finetuning chinese model: taide model
- Finetuning data from: [CFGPT](https://github.com/TongjiFinLab/CFGPT)

## Install Packages

In [1]:
"""
!pip install bitsandbytes==0.43.0
!pip install datasets==2.10.1
!pip install transformers==4.38.2
!pip install peft==0.9.0
!pip install sentencepiece==0.1.99
!pip install -U accelerate==0.28.0

!pip install colorama==0.4.6
!pip install -U datasets
"""
# Install protobuf with linux 

'\n!pip install bitsandbytes==0.43.0\n!pip install datasets==2.10.1\n!pip install transformers==4.38.2\n!pip install peft==0.9.0\n!pip install sentencepiece==0.1.99\n!pip install -U accelerate==0.28.0\n\n!pip install colorama==0.4.6\n!pip install -U datasets\n'

In [1]:
import os
import sys
import argparse
import json
import warnings
import logging
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, load_from_disk
import transformers, datasets
from peft import PeftModel
from colorama import *

from tqdm import tqdm
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import GenerationConfig
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training
)

In [2]:
seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [3]:
# 生成訓練資料
def generate_training_data(data_point):
    """
    (1) Goal:
        - This function is used to transform a data point (input and output texts) to tokens that our model can read

    (2) Arguments:
        - data_point: dict, with field "instruction", "input", and "output" which are all str

    (3) Returns:
        - a dict with model's input tokens, attention mask that make our model causal, and corresponding output targets

    (3) Example:
        - If you construct a dict, data_point_1, with field "instruction", "input", and "output" which are all str, you can use the function like this:
            formulate_article(data_point_1)

    """
    # construct full input prompt
    prompt = f"""\
[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem. 你是一個樂於助人的助手且擅長寫唐詩。
<</SYS>>

{data_point["instruction"]}
{data_point["input"]}
[/INST]"""
    # count the number of input tokens
    len_user_prompt_tokens = (
        len(
            tokenizer(
                prompt,
                truncation=True,
                max_length=CUTOFF_LEN + 1,
                padding="max_length",
            )["input_ids"]
        ) - 1
    )
    # transform input prompt into tokens
    full_tokens = tokenizer(
        prompt + " " + data_point["output"] + "</s>",
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )["input_ids"][:-1]
    return {
        "input_ids": full_tokens,
        "labels": [-100] * len_user_prompt_tokens
        + full_tokens[len_user_prompt_tokens:],
        "attention_mask": [1] * (len(full_tokens)),
    }

# 進行生成回覆的評估
def evaluate(instruction, generation_config, max_len, input="", verbose=True):
    """
    (1) Goal:
        - This function is used to get the model's output given input strings

    (2) Arguments:
        - instruction: str, description of what you want model to do
        - generation_config: transformers.GenerationConfig object, to specify decoding parameters relating to model inference
        - max_len: int, max length of model's output
        - input: str, input string the model needs to solve the instruction, default is "" (no input)
        - verbose: bool, whether to print the mode's output, default is True

    (3) Returns:
        - output: str, the mode's response according to the instruction and the input

    (3) Example:
        - If you the instruction is "ABC" and the input is "DEF" and you want model to give an answer under 128 tokens, you can use the function like this:
            evaluate(instruction="ABC", generation_config=generation_config, max_len=128, input="DEF")

    """
    # construct full input prompt
    prompt = f"""\
[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem. 你是一個樂於助人的助手且擅長寫唐詩。
<</SYS>>

{instruction}
{input}
[/INST]"""
    # 將提示文本轉換為模型所需的數字表示形式
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    # 使用模型進行生成回覆
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_len,
    )
    # 將生成的回覆解碼並印出
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        output = output.split("[/INST]")[1].replace("</s>", "").replace("<s>", "").replace("Assistant:", "").replace("Assistant", "").strip()
        if (verbose):
            print(output)

    return output

# 模型輸出
def eval_demo():
    instructions = [
        "你是一位資深金融投資者，請根據研報中的投資建議，給出適合該公司的投資評級(1到10分，10分最好，1分最差)。研報內容：",  
        "請你對這份研報中提到的可能存在的投資風險進行分析，特別關注市場波動、行業政策等對投資的潛在影響。研報內容：",
        ]
    input = [
        "投資要點：\n中國電影是覆蓋全產業鍊的電影龍頭公司，實控人是中影集團\n中國電影是國內唯一一家集電影製作、發行、放映和影視服務全產業鍊覆蓋的上市公司， 由中影集團及中國國際電視、央廣傳媒、長影集團、江蘇廣電、歌華有線、電廣傳媒、中國聯通 7 家公司共同發起設立。 目前中影集團是公司的控股股東和實際控制人， 擁有公司 67.36%的股份； 截止 2018 年三季報公司前十大股東持股比例為 76.41%， 持股集中度相對較高。\n公司發行和放映業務的實力強， 全產業鍊競爭力突出\n公司的經營業務全面覆蓋電影產業鍊的上中下游， 分別包括上游的影視製片製作業務、中游的電影發行營銷業務以及下游的電影放映、影視服務業務。其中在影視發行營銷業務方面，公司是國內兩家進口片發行商之一， 具有極高的政策壁壘和競爭力；在電影放映業務方面，公司控股的中影數字、中影星美、中影南方新乾線三家院線是國內院線領域的 TOP10， 呲合競爭力突出；電影服務業務中，公司擁有中國巨幕的研發、生產和銷售能力，此外還進行影院放映設備的銷售，受益於近年來國內新建影院的快速增長，公司的中國巨幕系統發展迅速。因此從布局上分析， 中國電影無論是在單項業務還是整體業務實力都在國內處於絕對龍頭的地位，競爭力和經營壁壘十分明顯。\n電影行業： 2019 年上映大片陣容豪華，驅動市場發展\n2018 年 1-11 月國內電影總票房收入 563.12 億元， 同比增長 10.96%， 整體表現基本符合預期。我們預計 2019 年全年票房繼續保持平穩增長， 進口片的相對競爭優勢更強，邏輯在於多部美國科幻電影陸續上映將為國內電影票房市場形成有力的支撐；同時 2019 年春節檔的眾多國產賀歲電影（如流浪地球等）也具備很強的競爭力，因此整體上我們認為 2019 年國內電影票房市場將繼續保持平穩增速水平，增速相較於 2018 年將有所提高， 院線和相關爆款影片的製片/發行方將更加受益。"\
        , 
        "本周行業觀點：格局重塑，全球化加速\n1、新能源汽車行業觀點：1、價、量短期弱勢，格局仍在重塑中。上周中游總體價格平穩，三元和隔膜價格出現一定的下跌：NCM523動力型三元材料主流價環比上周下跌1.9%；溼法基膜主流價環比上周下跌5%。臨近春節，由於短期淡季、去庫存影響疊加政策的變動窗口期，價格和出貨量預計短期保持弱勢。從中長期的趨勢看，三元和溼法隔膜的趨勢明顯，經歷這次價格低谷後，預計行業格局將重塑。2、國內龍頭公司加速融入全球高端供應鍊，關注全球化龍頭機會。我們看好新能源汽車未來全球化、高端化的投資機會，特斯拉產業鍊和高端鋰電產業鍊我們關注的重點，近期的進展有：1）先導智能正式供貨特斯拉，打開了國內裝備配套全球的空間，也加速了進一步的進口替代；2）格林美的鋰電上游材料進入韓國的供應體系，雖然只是一個長期的MOU，但也代表著國內產品的升級，還代表著2019年是國內中上游高端製造享受新能源汽車全球化紅利的開始。3、回收體系逐步建立，新機會來臨。我們的新能源汽車框架中關鍵一環就是能源體系的循環利用，動力電池的回收再利用就是核心；京津冀聯合發布《京津冀地區新能源汽車動力蓄電池回收利用試點實施方案》將推動行業發展，根據我們的調研，隨著供需體系的逐步建立，有技術和資源優勢的企業有望脫穎而出。\n2、光伏行業觀點：1、上游進口替代提速，新寡頭逐步形成。2019年我們看好上游硅料龍頭：1）去產能+進口替代持續進行。目前國內硅料在產企業18家，相較年初減少6家；新產能性價比提昇，新產能中滿足高品質單晶需求的占比在90%以上，這將加速進口替代。2）行業格局升級，新寡頭逐步形成。隨著通威永祥、新疆大全等優質產能投產，全球格局發生變化；尤其是通威股份的市場份額，其未來份額的提昇應該是最大的：隨著公司樂山年產5萬噸高純晶硅項目一期的正式投產，公司高純晶硅實際產能將達到8萬噸/年。2、淡季不淡，中游價格初漲。受益於全球市場的發展，2019年一季度行業需求較好。上周硅片、電池片價格都出現了上漲，其中硅片加上上漲在2%左右，電池片價格上漲在4%左右。展望一月份，由於春節放假與海外需求有一定的錯配，價格仍有望堅挺。3、儲能空間可以期待。隨著光伏等清潔能源占比的提昇，儲能將成為必需品；我國西北地區尤其是新疆、甘肅地區要進一步發展光伏，就存在著儲能的剛需，近期國家能源局復函同意甘肅省開展國家網域大規模電池儲能電站試驗示範工作，是一個較好的信號。"\
        ]
    
    # evaluate('以下是一首唐詩的第一句話，請用你的知識判斷並完成整首詩。', generation_config, max_len, tang, verbose = False))

    for input, instruct in zip(input, instructions):
        output = evaluate(instruct, generation_config, max_len, input, verbose = False)
        print(f'Instruction: \n {instruct} \n Input: {input}')
        print('===')
        print(f'Output: {output}')
        print('----------')


## Download model and inference before fine-tuning

The following code block takes about **10** minutes to run if you use the default setting, but it may vary depending on the condition of Colab.

In [4]:
""" 
model list
- 'taide/Llama3-TAIDE-LX-8B-Chat-Alpha1-4bit'
- 'taide/Llama3-TAIDE-LX-8B-Chat-Alpha1'  
- "./TAIDE-LX-7B-Chat"   
"""

model_name =  "./TAIDE-LX-7B-Chat"        # 設定想要用來進行fine-tune的模型，預設是使用TAIDE 7B的模型
#model_name = "MediaTek-Research/Breeze-7B-Instruct-v0_1"   # 若想選擇使用MediaTek Breeze 7B的模型，可以將這行最前面的 "#" 刪除，並把底下 "!" 開頭的兩行刪除

# If you want to use the TAIDE model, you should check out the TAIDE L Models Community License Agreement (https://drive.google.com/file/d/1FcUZjbUH6jr4xoCyAronN_slLgcdhEUd/view) first.
# Once you use it, it means you agree to the terms of the agreement.
# !wget -O taide_7b.zip "https://www.dropbox.com/scl/fi/harnetdwx2ttq1xt94rin/TAIDE-LX-7B-Chat.zip?rlkey=yzyf5nxztw6farpwyyildx5s3&st=s22mz5ao&dl=0"

# !unzip taide_7b.zip

In [6]:
# Set models
cache_dir = "./cache"

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# 從指定的模型名稱或路徑載入預訓練的語言模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    quantization_config=nf4_config,
    low_cpu_mem_usage = True
)

# 創建 tokenizer 並設定結束符號 (eos_token)
logging.getLogger('transformers').setLevel(logging.ERROR)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_eos_token=True,
    cache_dir=cache_dir,
    quantization_config=nf4_config
)
tokenizer.pad_token = tokenizer.eos_token

# 設定模型推理時需要用到的decoding parameters
max_len = 128
generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.1,
    num_beams=1,
    top_p=0.3,
    no_repeat_ngram_size=3,
    pad_token_id=2,
)

Loading checkpoint shards:  67%|██████▋   | 2/3 [00:12<00:06,  6.45s/it]

## Inference before Fine-tuning
Let's first see what our model can do without fine-tuning.

In [ ]:
eval_demo()

## Set Hyperarameters for Fine-tuning



In [5]:
""" It is highly recommended you try to play around this hyperparameter """

num_train_data = 1040 # 設定用來訓練的資料數量，可設置的最大值為5000。在大部分情況下會希望訓練資料盡量越多越好，這會讓模型看過更多樣化的詩句，進而提升生成品質，但是也會增加訓練的時間
                      # 使用預設參數(1040): fine-tuning大約需要25分鐘，完整跑完所有cell大約需要50分鐘
                      # 使用最大值(5000): fine-tuning大約需要100分鐘，完整跑完所有cell大約需要120分鐘

In [6]:
""" You may want (but not necessarily need) to change some of these hyperparameters """

output_dir = "taide-finetune-output"  # 設定作業結果輸出目錄 (如果想要把作業結果存在其他目錄底下可以修改這裡，強烈建議存在預設值的子目錄下，也就是Google Drive裡)
ckpt_dir = "./taide-checkpoints-exp1" # 設定model checkpoint儲存目錄 (如果想要將model checkpoints存在其他目錄下可以修改這裡)
num_epoch = 50  # 設定訓練的總Epoch數 (數字越高，訓練越久，若使用免費版的colab需要注意訓練太久可能會斷線)
LEARNING_RATE = 3e-4  # 設定學習率

In [7]:
""" It is recommmended NOT to change codes in this cell """

cache_dir = "./cache"  # 設定快取目錄路徑
from_ckpt = True  # 是否從checkpoint載入模型的權重，預設為否
ckpt_name = "taide-checkpoints-exp1/checkpoint-50"  # 從特定checkpoint載入權重時使用的檔案名稱，預設為無
dataset_dir = "./data/finetune-Text2Text-CH/CFGPT/data/traditional/data.json"  # 設定資料集的目錄或檔案路徑
logging_steps = 1  # 定義訓練過程中每隔多少步驟輸出一次訓練誌
save_steps = 50  # 定義訓練過程中每隔多少步驟保存一次模型
save_total_limit = 3  # 控制最多保留幾個模型checkpoint
report_to = None  # 設定上報實驗指標的目標，預設為無
MICRO_BATCH_SIZE = 4  # 定義微批次的大小
BATCH_SIZE = 16  # 定義一個批次的大小
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE  # 計算每個微批次累積的梯度步數
CUTOFF_LEN = 256  # 設定文本截斷的最大長度
LORA_R = 8  # 設定LORA（Layer-wise Random Attention）的R值
LORA_ALPHA = 16  # 設定LORA的Alpha值
LORA_DROPOUT = 0.05  # 設定LORA的Dropout率
VAL_SET_SIZE = 0  # 設定驗證集的大小，預設為無
WARMUP_STEP = 50
TARGET_MODULES = ["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj", "v_proj"] # 設定目標模組，這些模組的權重將被保存為checkpoint
device_map = "auto"  # 設定設備映射，預設為"auto"
world_size = int(os.environ.get("WORLD_SIZE", 1))  # 獲取環境變數"WORLD_SIZE"的值，若未設定則預設為1
ddp = world_size != 1  # 根據world_size判斷是否使用分散式數據處理(DDP)，若world_size為1則不使用DDP
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size

## Training: Start Fine-tuning

In [12]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [13]:
""" It is recommmended NOT to change codes in this cell """

# create the output directory you specify
os.makedirs(output_dir, exist_ok = True)
os.makedirs(ckpt_dir, exist_ok = True)

# 根據 from_ckpt 標誌，從 checkpoint 載入模型權重
if from_ckpt:
    print('Load model form ckpt')
    model = PeftModel.from_pretrained(model, ckpt_name)

# 將模型準備好以使用 INT8 訓練
model = prepare_model_for_int8_training(model)

# 使用 LoraConfig 配置 LORA 模型
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

# 將 tokenizer 的 padding token 設定為 0
tokenizer.pad_token_id = 0

# 載入並處理訓練數據
with open(dataset_dir, "r", encoding = "utf-8") as f:
    data_json = json.load(f)
with open("tmp_dataset.json", "w", encoding = "utf-8") as f:
    json.dump(data_json[:num_train_data], f, indent = 2, ensure_ascii = False)

data = load_dataset('json', data_files="tmp_dataset.json", download_mode="force_redownload")

# 將訓練數據分為訓練集和驗證集（若 VAL_SET_SIZE 大於 0）
if VAL_SET_SIZE > 0:
    train_val = data["train"].train_test_split(
        test_size=VAL_SET_SIZE, shuffle=True, seed=42
    )
    train_data = train_val["train"].shuffle().map(generate_training_data)
    val_data = train_val["test"].shuffle().map(generate_training_data)
else:
    train_data = data['train'].shuffle().map(generate_training_data)
    val_data = None

# 使用 Transformers Trainer 進行模型訓練
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=WARMUP_STEP,
        num_train_epochs=num_epoch,
        learning_rate=LEARNING_RATE,
        fp16=True,  # 使用混合精度訓練
        logging_steps=logging_steps,
        save_strategy="steps",
        save_steps=save_steps,
        output_dir=ckpt_dir,
        save_total_limit=save_total_limit,
        ddp_find_unused_parameters=False if ddp else None,  # 是否使用 DDP，控制梯度更新策略
        report_to=report_to,
        # eval_steps=logging_steps,
        # disable_tqdm=False,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    # compute_matrix = ...
)

# 禁用模型的 cache 功能
model.config.use_cache = False

# 若使用 PyTorch 2.0 版本以上且非 Windows 系統，進行模型編譯
if torch.__version__ >= "2" and sys.platform != 'win32':
    model = torch.compile(model)

# 開始模型訓練
trainer.train() 

# 將訓練完的模型保存到指定的目錄中
model.save_pretrained(ckpt_dir)

# 印出訓練過程中可能的缺失權重的警告信息
print("\n If there's a warning about missing keys above, please disregard :)")

Load model form ckpt


Generating train split: 46 examples [00:00, 1706.48 examples/s]
Map: 100%|██████████| 46/46 [00:00<00:00, 358.58 examples/s]


{'loss': 4.0647, 'grad_norm': 2.063822031021118, 'learning_rate': 5.999999999999999e-06, 'epoch': 0.33}
{'loss': 4.3169, 'grad_norm': 1.8890256881713867, 'learning_rate': 1.1999999999999999e-05, 'epoch': 0.67}
{'loss': 3.9286, 'grad_norm': 1.8754812479019165, 'learning_rate': 1.7999999999999997e-05, 'epoch': 1.0}
{'loss': 4.0891, 'grad_norm': 1.9078642129898071, 'learning_rate': 2.3999999999999997e-05, 'epoch': 1.33}
{'loss': 3.9681, 'grad_norm': 1.841944694519043, 'learning_rate': 2.9999999999999997e-05, 'epoch': 1.67}
{'loss': 4.0903, 'grad_norm': 1.5156561136245728, 'learning_rate': 3.5999999999999994e-05, 'epoch': 2.0}
{'loss': 3.772, 'grad_norm': 1.3767019510269165, 'learning_rate': 4.2e-05, 'epoch': 2.33}
{'loss': 3.8574, 'grad_norm': 1.453385829925537, 'learning_rate': 4.7999999999999994e-05, 'epoch': 2.67}
{'loss': 3.8943, 'grad_norm': 1.5729848146438599, 'learning_rate': 5.399999999999999e-05, 'epoch': 3.0}
{'loss': 3.7361, 'grad_norm': 1.343515157699585, 'learning_rate': 5.99

##  Testing

In [8]:
# find all available checkpoints
ckpts = []
for ckpt in os.listdir(ckpt_dir):
    if (ckpt.startswith("checkpoint-")):
        ckpts.append(ckpt)

# list all the checkpoints
ckpts = sorted(ckpts, key = lambda ckpt: int(ckpt.split("-")[-1]))
print("all available checkpoints:")
print(" id: checkpoint name")
for (i, ckpt) in enumerate(ckpts):
    print(f"{i:>3}: {ckpt}")


all available checkpoints:
 id: checkpoint name
  0: checkpoint-50


In [9]:
id_of_ckpt_to_use = -1  # 要用來進行推理的checkpoint的id(對應上一個cell的輸出結果)
                        # 預設值-1指的是上列checkpoints中的"倒數"第一個，也就是最後一個checkpoint
                        # 如果想要選擇其他checkpoint，可以把-1改成有列出的checkpoint id中的其中一個

ckpt_name = os.path.join(ckpt_dir, ckpts[id_of_ckpt_to_use])

In [10]:
# 你可以在這裡調整decoding parameter，decoding parameter的詳細解釋請見homework slides
max_len = 128   # 生成回復的最大長度
temperature = 0.1  # 設定生成回覆的隨機度，值越小生成的回覆越穩定
top_p = 0.3  # Top-p (nucleus) 抽樣的機率閾值，用於控制生成回覆的多樣性
top_k = 5 # 調整Top-k值，以增加生成回覆的多樣性和避免生成重複的詞彙

In [11]:
test_data_path = "GenAI-Hw5/Tang_testing_data.json"
output_path = os.path.join(output_dir, "results.txt")

cache_dir = "./cache"  # 設定快取目錄路徑
seed = 42  # 設定隨機種子，用於重現結果
no_repeat_ngram_size = 3  # 設定禁止重複 Ngram 的大小，用於避免生成重複片段

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# 使用 tokenizer 將模型名稱轉換成模型可讀的數字表示形式
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    quantization_config=nf4_config
)

# 從預訓練模型載入模型並設定為 8 位整數 (INT8) 模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=nf4_config,
    device_map={'': 0},  # 設定使用的設備，此處指定為 GPU 0
    cache_dir=cache_dir
)

# 從指定的 checkpoint 載入模型權重
model = PeftModel.from_pretrained(model, ckpt_name, device_map={'': 0})

# 設定生成配置，包括隨機度、束搜索等相關參數
generation_config = GenerationConfig(
    do_sample=True,
    temperature=temperature,
    num_beams=1,
    top_p=top_p,
    # top_k=top_k,
    no_repeat_ngram_size=no_repeat_ngram_size,
    pad_token_id=2
)


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Loading checkpoint shards: 100%|██████████| 3/3 [00:18<00:00,  6.16s/it]


In [12]:
eval_demo()

Instruction: 
 你是一位資深金融投資者，請根據研報中的投資建議，給出適合該公司的投資評級(1到10分，10分最好，1分最差)。研報內容： 
 Input: 投資要點：
中國電影是覆蓋全產業鍊的電影龍頭公司，實控人是中影集團
中國電影是國內唯一一家集電影製作、發行、放映和影視服務全產業鍊覆蓋的上市公司， 由中影集團及中國國際電視、央廣傳媒、長影集團、江蘇廣電、歌華有線、電廣傳媒、中國聯通 7 家公司共同發起設立。 目前中影集團是公司的控股股東和實際控制人， 擁有公司 67.36%的股份； 截止 2018 年三季報公司前十大股東持股比例為 76.41%， 持股集中度相對較高。
公司發行和放映業務的實力強， 全產業鍊競爭力突出
公司的經營業務全面覆蓋電影產業鍊的上中下游， 分別包括上游的影視製片製作業務、中游的電影發行營銷業務以及下游的電影放映、影視服務業務。其中在影視發行營銷業務方面，公司是國內兩家進口片發行商之一， 具有極高的政策壁壘和競爭力；在電影放映業務方面，公司控股的中影數字、中影星美、中影南方新乾線三家院線是國內院線領域的 TOP10， 呲合競爭力突出；電影服務業務中，公司擁有中國巨幕的研發、生產和銷售能力，此外還進行影院放映設備的銷售，受益於近年來國內新建影院的快速增長，公司的中國巨幕系統發展迅速。因此從布局上分析， 中國電影無論是在單項業務還是整體業務實力都在國內處於絕對龍頭的地位，競爭力和經營壁壘十分明顯。
電影行業： 2019 年上映大片陣容豪華，驅動市場發展
2018 年 1-11 月國內電影總票房收入 563.12 億元， 同比增長 10.96%， 整體表現基本符合預期。我們預計 2019 年全年票房繼續保持平穩增長， 進口片的相對競爭優勢更強，邏輯在於多部美國科幻電影陸續上映將為國內電影票房市場形成有力的支撐；同時 2019 年春節檔的眾多國產賀歲電影（如流浪地球等）也具備很強的競爭力，因此整體上我們認為 2019 年國內電影票房市場將繼續保持平穩增速水平，增速相較於 2018 年將有所提高， 院線和相關爆款影片的製片/發行方將更加受益。
===
Output: 評級：8
----------
Instruction: 
 請你對這份研報中提到的可能存在的投資風險進行分析，特別關注市場波動、行業政策等對投資的潛在影響。研報內容： 
